In [1]:
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torchvision
import torch 
from torch import nn

from networks.ae import AutoEncoder
from utils.train_options import *
from networks.patchgan import *
from networks.deep_unet import *
from networks.unet import *
from networks.unet64 import *
from networks.attunet import *
from datasets.KonIQDataset import *
from datasets.TestDataset import * 
from datasets.SynthTextDataset import *
from datasets.ValidDataset import *

In [36]:
device = 'cuda'
gen = AttU_Net(img_ch=3, output_ch=3)
gen.load_state_dict(torch.load('/home/igeorvasilis/thesis_src/checkpoints/latest_Gatt(L1c).pth'))
#gen = UNet()
#gen.load_state_dict(torch.load('/home/igeorvasilis/thesis_src/checkpoints/latest_G_all_20.pth'))
gen = gen.to(device)

In [29]:
transform = torchvision.transforms.Compose([
		torchvision.transforms.ToTensor(),
		torchvision.transforms.Resize((256,512))
])

dataset = ValidDataset('/home/igeorvasilis/sdb/KoniQ_dataset/A_B/test', transform=transform)
test_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)

In [4]:
transform = torchvision.transforms.Compose([
		torchvision.transforms.ToTensor(),
		torchvision.transforms.Resize((256,256))
])


TEST_ROOT = "/home/igeorvasilis/diploma thesis/noise2noise-pytorch-master/data/test_out/test"
test_dataset = TestDataset(TEST_ROOT, transform=transform, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32)

['test(17).png', 'test(18).png', 'test(19).png', 'test(20).png', 'test(21).png', 'test(22).png', 'test(23).png', 'test(24).png', 'test(25).png', 'test(26).png', 'test(27).png', 'test(28).png', 'test(29).png', 'test(30).png', 'test(31).png', 'test(32).png', 'test1.png', 'test10.png', 'test11.png', 'test12.png', 'test13.png', 'test14.png', 'test15.png', 'test16.png', 'test2.png', 'test3.png', 'test4.png', 'test5.png', 'test6.png', 'test7.png', 'test8.png', 'test9.png']


In [5]:
for batch_id, (condition) in enumerate(test_loader):
    condition = condition.to(device)
    with torch.no_grad():
        fake = gen(condition)

In [5]:
def PSNR(img1, img2):
    """Peak Signal to Noise Ratio
    img1 and img2 have range [0, 255]"""
    img1 *= 255.0
    img2 *= 255.0 
    
    mse = torch.mean((img1 - img2) ** 2)
    return 20 * torch.log10(255.0 / torch.sqrt(mse))

In [37]:
from skimage.measure import compare_ssim

psnr_score = 0.0
ssim_score = 0.0
for batch_id, (input_image) in enumerate(test_loader):
    
    input_image = input_image[0].to(device)

    image_width = input_image.shape[2]
    condition = input_image[:, :, :image_width // 2].unsqueeze(0)
    target = input_image[:, :, image_width // 2:].unsqueeze(0)
    #condition = input_image[:, :, :input_image.shape[2] // 2].unsqueeze(0)
    #target = input_image[:, :, input_image.shape[2] // 2 :].unsqueeze(0)
    
    with torch.no_grad():
        fake = gen(condition)
    psnr_score += PSNR(fake, target) / len(test_loader)

    fake = fake.to(device='cpu')
    condition = condition.to(device='cpu')
    target = target.to(device='cpu')

    #print(target.shape, condition.shape)
    save_image(target, 'o.png')

    target = target[0].permute(1, 2, 0).numpy()
    fake = fake[0].permute(1, 2, 0).numpy()
    condition = condition[0].permute(1, 2, 0).numpy()
    
    fake = cv2.cvtColor(fake, cv2.COLOR_BGR2GRAY)
    target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    condition = cv2.cvtColor(condition, cv2.COLOR_BGR2GRAY)

    (score, diff) = compare_ssim(fake, target, full=True)
    diff = (diff * 255).astype("uint8")
    
    ssim_score += score / len(test_loader)

psnr_score, ssim_score

    
    

ipykernel_launcher:34: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.


(tensor(19.9091, device='cuda:0'), 0.664220593055369)